# Задача 1: вычислить $\sqrt [k]a$
#### - методом бисекции
#### - методом Ньютона

In [1]:
import random

Сразу отмечу, что я буду рассматривать $a >= 0$. Если же $a < 0$, то если $k$ - чётно, то решений нет, а если $k$ - нечётно, то достаточно взять корень k-й степени из $|a|$ и умножить на $(-1)$

### Метод бисекции

In [2]:
def bisection(f, l, r, eps):
    if f(l) == 0:
        return l
    if f(r) == 0:
        return r
    assert(eps > 0)
    assert(f(l) * f(r) < 0)
    assert(l <= r)
    while r - l > eps:
        m = (l + r) / 2
        if f(m) == 0:
            return m
        if f(l) * f(m) < 0:
            r = m
        else:
            l = m
    return (l + r) / 2

In [3]:
eps = 10**(-6)
for r in range(2, 11):
    k = r
    a = random.randint(1, 1000)
    res = bisection(lambda x: (x ** k - a), 0, max(a, 1), eps)
    diff = res - a**(1/k)
    print(f"a = {a}, k = {k}, a**(1/k) = {a**(1/k)}, method's result = {res}, diff = {diff}, eps = {eps}")

a = 472, k = 2, a**(1/k) = 21.72556098240043, method's result = 21.725560776889324, diff = -2.0551110679889462e-07, eps = 1e-06
a = 241, k = 3, a**(1/k) = 6.223084253206058, method's result = 6.22308418340981, diff = -6.979624789238414e-08, eps = 1e-06
a = 376, k = 4, a**(1/k) = 4.403489460605681, method's result = 4.403489552438259, diff = 9.183257798639488e-08, eps = 1e-06
a = 670, k = 5, a**(1/k) = 3.6746413735593393, method's result = 3.674641498364508, diff = 1.2480516886625992e-07, eps = 1e-06
a = 268, k = 6, a**(1/k) = 2.5391545300093323, method's result = 2.539154790341854, diff = 2.6033252176205224e-07, eps = 1e-06
a = 22, k = 7, a**(1/k) = 1.5551585367634633, method's result = 1.5551588237285614, diff = 2.869650981285332e-07, eps = 1e-06
a = 358, k = 8, a**(1/k) = 2.0856209388261213, method's result = 2.0856210235506296, diff = 8.472450829799527e-08, eps = 1e-06
a = 533, k = 9, a**(1/k) = 2.008952599693231, method's result = 2.0089524844661355, diff = -1.1522709542077791e-07,

### Метод Ньютона

In [4]:
def newton_method(f, df, x, eps):
    x_prev = x + 2 * eps
    while abs(x_prev - x) > eps:
        x_prev = x
        x = x_prev - f(x_prev)/df(x_prev)
    return x

In [5]:
eps = 10**(-6)
for r in range(2, 11):
    k = r
    a = random.randint(1, 1000)
    res = newton_method(lambda x: (x ** k - a), lambda x: (k * (x ** (k - 1))), a, eps)
    diff = res - a**(1/k)
    print(f"a = {a}, k = {k}, a**(1/k) = {a**(1/k)}, method's result = {res}, diff = {diff}, eps = {eps}")

a = 975, k = 2, a**(1/k) = 31.22498999199199, method's result = 31.224989991991993, diff = 3.552713678800501e-15, eps = 1e-06
a = 357, k = 3, a**(1/k) = 7.093970944750709, method's result = 7.093970944750712, diff = 2.6645352591003757e-15, eps = 1e-06
a = 292, k = 4, a**(1/k) = 4.133764324515255, method's result = 4.133764324515354, diff = 9.947598300641403e-14, eps = 1e-06
a = 924, k = 5, a**(1/k) = 3.9186312202916507, method's result = 3.9186312202916507, diff = 0.0, eps = 1e-06
a = 184, k = 6, a**(1/k) = 2.384897054319773, method's result = 2.3848970543200805, diff = 3.0730973321624333e-13, eps = 1e-06
a = 371, k = 7, a**(1/k) = 2.3283776167657475, method's result = 2.3283776167669235, diff = 1.1759482276829658e-12, eps = 1e-06
a = 740, k = 8, a**(1/k) = 2.2837784204514677, method's result = 2.2837784204514677, diff = 0.0, eps = 1e-06
a = 915, k = 9, a**(1/k) = 2.1332747267304524, method's result = 2.1332747267304524, diff = 0.0, eps = 1e-06
a = 958, k = 10, a**(1/k) = 1.98671948367

# Задание 2

Первые две части стоит рассматривать в совокупности. Давайте посмотрим на $p(x)$. Какие у него могут быть корни?

Во-первых, корни нечетной кратности - они будут между нулями производной (т.е. между экстремумами исходной функции) или также исходные границы. Что будет соответствовать нашему разбиению (т.е. пункт а) - возращаемое значению функцией lrs(...).

Во-вторых, корни чётной кратности. Они не будут подходить под условие изолированности из первого пункта (т.к. с двух сторон от корня будет либо положительность, либо отрицательность). Поэтому я решил, что не стоит вносить их в функцию lrs. Однако их нужно безусловно также искать - это будут точки экстремума (ну и теоретически l и r). Давайте всех их и проверим (функция even_roots).

In [6]:
from numpy import polynomial
import numpy as np

In [7]:
def filter_eps(a, eps):
    if len(a) == 0:
        return []
    ans = [a[0]]
    last = a[0]
    for i in range(1, len(a)):
        if abs(a[i] - last) >= 2*eps:
            ans.append(a[i])
            last = a[i]
    return ans

In [8]:
def lrs(p, l, r, eps):
    droots = roots(p.deriv(), l, r, eps)
    ans = []
    if len(droots) > 0:
        if p(l) * p(droots[0]) < 0:
            ans.append((l, droots[0]))
        for i in range(1, len(droots)):
            if p(droots[i-1]) * p(droots[i]) < 0:
                ans.append((droots[i-1], droots[i]))
        if p(r) * p(droots[-1]) < 0:
            ans.append((droots[-1], r))
    else:
        if p(l) * p(r) < 0:
            ans.append((l, r))
    return ans

In [9]:
def even_roots(p, l, r, eps):
    droots = roots(p.deriv(), l, r, eps)
    ans = []
    if abs(p(l)) < eps:
        ans.append(l)
    for dr in droots:
        if abs(p(dr)) < eps:
            ans.append(dr)
    if abs(p(r)) < eps:
        ans.append(r)
    return ans

In [10]:
def roots(p, l, r, eps):
    if p.degree() == 0:
        return []
    ans = []
    for (l_, r_) in lrs(p, l, r, eps):
        ans.append(bisection(p, l_, r_, eps))
    ans.extend(even_roots(p, l, r, eps))
    return filter_eps(sorted(ans), eps)

Таким образом мы решили первые два пункта.

Какие потенциальные точки минимума? Это все экстремумы (т.е. нули производной и l, r).

In [11]:
def minimum(p, l, r, eps):
    ans = 0
    if p(l) < p(r):
        ans = p(l)
    else:
        ans = p(r)
    for x in roots(p.deriv(), l, r, eps):
        if p(x) < ans:
            ans = p(x)
    return ans

In [12]:
eps = 10**(-8)

p2 = polynomial.Polynomial(coef=(0.53, -2.42, 0.3))
p2_roots_from_wolfram = [0.225301, 7.84137]
p2_min_from_wolfram = -4.3503
print(f"roots: {roots(p2, -100, 100, eps)}, roots from wolframalpha: {p2_roots_from_wolfram}\nminimum: {minimum(p2, -100, 100, eps)}, minimum from wolfram: {p2_min_from_wolfram}")

p3 = polynomial.Polynomial(coef=(1, 1, 1, 1))
p3_roots_from_wolfram = [-1]
p3_min_from_wolfram = -909
print(f"roots: {roots(p3, -10, 10, eps)}, roots from wolframalpha: {p3_roots_from_wolfram}\nminimum: {minimum(p3, -10, 10, eps)}, minimum from wolfram: {p3_min_from_wolfram}")

p4 = polynomial.Polynomial(coef=(1, -1, 1, -1, -1))
p4_roots_from_wolfram = [-1.92756, 0.774804]
p4_min_from_wolfram = -100990099
print(f"roots: {roots(p4, -100, 100, eps)}, roots from wolframalpha: {p4_roots_from_wolfram}\nminimum: {minimum(p4, -100, 100, eps)}, minimum from wolfram: {p4_min_from_wolfram}")

p5 = polynomial.Polynomial(coef=(1, 2, 3, 4, 5, 6))
p5_roots_from_wolfram = [-0.670332]
p5_min_from_wolfram = -59503970199
print(f"roots: {roots(p5, -100, 100, eps)}, roots from wolframalpha: {p5_roots_from_wolfram}\nminimum: {minimum(p5, -100, 100, eps)}, minimum from wolfram: {p5_min_from_wolfram}")

roots: [0.22530088893615463, 7.841365777955575], roots from wolframalpha: [0.225301, 7.84137]
minimum: -4.3503333333333325, minimum from wolfram: -4.3503
roots: [-1.0000000009313226], roots from wolframalpha: [-1]
minimum: -909.0, minimum from wolfram: -909
roots: [-1.9275619776851183, 0.774804110483194], roots from wolframalpha: [-1.92756, 0.774804]
minimum: -100990099.0, minimum from wolfram: -100990099
roots: [-0.6703320454107597], roots from wolframalpha: [-0.670332]
minimum: -59503970199.0, minimum from wolfram: -59503970199


# Задание 3
$f(x) = e^{ax} + e^{-bx} + c(x-d)^2$
Требуется найти минимум $f(x)$ при условии, что $a, b, c > 0$

Давайте рассмотрим $f(x)$. Она выпукла, т.к. все слагаемые выпуклы, а линейная комбинация выпуклых функций - выпукла.

Давайте тогда посчитаем производную $f$:
$f'(x) = a\cdot e^{ax} - b\cdot e^{-bx} + 2c(x-d)$

Посмотрим сразу на вторую производную:
$f''(x) = a^2\cdot e^{ax} + b^2\cdot e^{-bx} + 2c$.

Заметим, что при наших ограничениях на $a, b, c$ имеем $f''(x) > 0$ (кроме того очевидно, что $f, f', f''$ непрерывны). Это значит, что $f'(x)$ монотонно возрастает. А это значит в свою очередь, что $f(x)$ имеет один экстремум (в точке, где $f'(x) = 0$), а т.к. $f'(x)$ - возрастающая, то это минимум. Значит нам необходимо найти ноль производной.

In [13]:
from numpy import exp

eps = 10**(-9)
def task3_funcs(a, b, c, d):
    f = lambda x: exp(a*x) + exp(-b*x) + c*(x - d)**2
    df = lambda x: a * (exp(a*x)) - b * (exp(-b*x)) + 2 * c * (x - d)
    d2f = lambda x: a * a * (exp(a*x)) + b * b * (exp(-b*x)) + 2 * c
    return (f, df, d2f)

In [14]:
def ternarny_search(f, l, r, eps):
    while (r - l) > eps:
        ml = l + (r - l) / 3
        mr = r - (r - l) / 3
        if f(ml) > f(mr):
            l = ml
        else:
            r = mr
    return (r + l) / 2

In [15]:
(f, df, d2f) = task3_funcs(1, 2, 3, 4)

r = 1
while df(r) <= 0:
    r *= 2
l = -1
while df(l) >= 0:
    l *= 2
bisection_res = bisection(df, l, r, eps)
print(f"Бисекция: x = {bisection_res}, f(x) = {f(bisection_res)}")

Бисекция: x = 2.3152837297529913, f(x) = 18.652352027148623


In [16]:
newton_res = newton_method(df, d2f, 0, eps = 10**(-18))
print(f"Метод Ньютона: x = {newton_res}, f(x) = {f(newton_res)}")

Метод Ньютона: x = 2.315283729985808, f(x) = 18.652352027148623


In [17]:
ternarny_res = ternarny_search(f, l, r, eps)
print(f"Тернарный поиск: x = {ternarny_res}, f(x) = {f(ternarny_res)}")

Тернарный поиск: x = 2.315283714363277, f(x) = 18.652352027148623
